<a href="https://colab.research.google.com/github/MarshaGomez/DNN-Sketches-image-analysis/blob/main/Code/Analysis_By_Class.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
import numpy as np
import time
import itertools
import os, shutil
import tensorflow as tf
import matplotlib.pyplot as plt

from random import random
from ipywidgets import Image
from numpy.linalg import norm
from IPython.display import display
from keras.models import Model
from sklearn.metrics import classification_report
from keras.callbacks import EarlyStopping, ModelCheckpoint

# tensorflow version 2.4.0
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.inception_v3 import preprocess_input, InceptionV3

In [ ]:
zip_path = '/content/gdrive/Shareddrives/COMPUTER_VISION/MIM_zipped.zip'
!cp "{zip_path}" .
!unzip -q MIM_zipped.zip
!rm MIM_zipped.zip

replace content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV/mirflickr/data/txt/sunset.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: N



In [ ]:
test_images_per_class = 20 # le immagini sono al massimo 80
train_images_per_class = 80 - test_images_per_class

In [ ]:
BASE_DIR = "/content/content/gdrive/Shareddrives/COMPUTER_VISION/MIRCV"
SKETCHES_DIR = os.path.join(BASE_DIR, "sketches")
MIRFLICKR_DIR = os.path.join(BASE_DIR, "mirflickr/mirflickr25k")

print(os.listdir(os.path.join(SKETCHES_DIR, "png")))
def copytree(src, dst, symlinks=False, ignore=None):
    for item in os.listdir(src):
        s = os.path.join(src, item)
        d = os.path.join(dst, item)
        if os.path.isdir(s):
            shutil.copytree(s, d, symlinks, ignore)
        else:
            shutil.copy2(s, d)
      
copytree(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/all')

try:
  os.rename(f'{SKETCHES_DIR}/png', f'{SKETCHES_DIR}/train')
except:
  print('already renamed')

if not os.path.exists(f'{SKETCHES_DIR}/test'):
  os.mkdir(f'{SKETCHES_DIR}/test')
  for class_folder in os.listdir(f'{SKETCHES_DIR}/train'):
    if os.path.isdir(f'{SKETCHES_DIR}/train/{class_folder}'):
      images = os.listdir(f'{SKETCHES_DIR}/train/{class_folder}')
      os.mkdir(f'{SKETCHES_DIR}/test/{class_folder}')
      for i in range(test_images_per_class):
        os.rename(f'{SKETCHES_DIR}/train/{class_folder}/{images[i]}', f'{SKETCHES_DIR}/test/{class_folder}/{images[i]}')

['rooster', 'head-phones', 'door handle', 'flashlight', 'panda', 'bottle opener', 'revolver', 'binoculars', 'piano', 'lightbulb', 'saxophone', 'snowboard', 'giraffe', 'filelist.txt', 'scissors', 'rabbit', 'pear', 'ship', 'helicopter', 'van', 'barn', 'tiger', 'dragon', 'ice-cream-cone', 'wheel', 'dolphin', 'cell phone', 'tree', 'squirrel', 'speed-boat', 'wheelbarrow', 'screwdriver', 'rifle', 'arm', 'bee', 'house', 'penguin', 'blimp', 'pretzel', 'motorbike', 'mosquito', 'hourglass', 'moon', 'spoon', 'pig', 'pizza', 'rollerblades', 'ant', 'cabinet', 'car (sedan)', 'book', 'bed', 'sailboat', 'candle', 'tooth', 'train', 'strawberry', 'palm tree', 'streetlight', 'present', 'angel', 'envelope', 'table', 'eye', 'bathtub', 'lion', 'fan', 'crocodile', 'castle', 'keyboard', 'crown', 'flying bird', 'human-skeleton', 'octopus', 'laptop', 'nose', 'parrot', 'umbrella', 'bookshelf', 'sponge bob', 'satellite', 'couch', 'bicycle', 'stapler', 'kangaroo', 'door', 'ear', 'mug', 'hot-dog', 'suv', 'fish', 'k

FileExistsError: ignored

In [ ]:
class NO_INDEX:
  def __init__(self):
    self.features = None
    self.ids = None
    self.labels = None

  def insert(self, features, ids, labels):
    assert features.shape[0] == len(ids), "mismatch between ids length and features"
    assert len(labels) == len(ids), "mismatch between ids length and labels"
    self.features = features
    self.ids = ids
    self.labels = labels
  
  def query(self, features, top_k, mode = 'euclidean', return_cost = False):
    assert mode in ['similarity', 'euclidean'], "mode must be similarity or euclidean"
    top_k += 1
    if mode == 'euclidean':
      dist = norm(self.features - np.array(features), axis=1)
      idx_partitioned = np.argpartition(dist, top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0] - 1)
      k = {}
      # qua è più comodo avere array 1- dimensionali
      k['ids'] = self.ids[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['labels'] = self.labels[idx_partitioned].flatten()[:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      k['distances'] = dist[idx_partitioned][:top_k - 1 if dist.shape[0] - 1 > top_k - 1 else dist.shape[0]]
      
      idx_sorted = np.argsort(k['distances'])
      idx_sorted = idx_sorted
      k['distances'] = k['distances'][idx_sorted]
      k['ids'] = k['ids'][idx_sorted]
      k['labels'] = k['labels'][idx_sorted]
      if return_cost:
        return (k, dist.size)
      return k
    sim = np.sum(self.features * np.array(features), axis=1) / (norm(self.features, axis=1) * norm(np.array([features]), axis=1))
    idx_partitioned = np.argpartition(sim, -(top_k - 1) if sim.shape[0] - 1 > top_k - 1 else sim.shape[0] - 1)
    k = {}
    # qua è più comodo avere array 1- dimensionali
    if sim.shape[0] - 1 > top_k - 1:  
      idx_partitioned = idx_partitioned[-(top_k - 1):]

    k['ids'] = self.ids[idx_partitioned].flatten()
    k['labels'] = self.labels[idx_partitioned].flatten()
    k['similarities'] = sim[idx_partitioned]
    idx_sorted = np.argsort(k['similarities'])[::-1]
    idx_sorted = idx_sorted
    k['similarities'] = k['similarities'][idx_sorted]
    k['ids'] = k['ids'][idx_sorted]
    k['labels'] = k['labels'][idx_sorted]
    if return_cost:
      return (k, sim.size)
    return k
    

In [ ]:
IMG_HEIGHT = 299
IMG_WIDTH = 299
INPUT_SHAPE = (IMG_WIDTH, IMG_HEIGHT, 3)
BATCH_SIZE = 64

In [ ]:
#yields batches of images from the subdirectories 
#found in the sketches directory [class_0 ... class_250]
#together with class labels.

#images normalization 
sketches_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
mirflickr_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

sketches_generator = sketches_datagen.flow_from_directory(
        # This is the target directory
        SKETCHES_DIR + '/all',
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

mirflickr_generator = mirflickr_datagen.flow_from_directory(
        # This is the target directory
        MIRFLICKR_DIR,
        shuffle=False,
        target_size=(IMG_HEIGHT, IMG_WIDTH),
        batch_size=BATCH_SIZE)

Found 20000 images belonging to 250 classes.
Found 25000 images belonging to 1 classes.


In [ ]:
#Extracting features using the pretrained convolutional base 

def extract_features(extractor, generator, sample_count, dim=2048):
  features = np.zeros((sample_count, dim)) #extractor output shape 
  i = 0
  for inputs_batch, labels_batch in generator:
    start = time.time()
    features_batch = extractor.predict(inputs_batch)
    start = time.time()
    if (i + 1) * BATCH_SIZE > sample_count:
      features[i * BATCH_SIZE : sample_count , :] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : sample_count]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    else:
      features[i * BATCH_SIZE : (i + 1) * BATCH_SIZE, : ] = features_batch
      assert np.array_equal(np.argmax(labels_batch, axis = 1), generator.labels[i * BATCH_SIZE : (i + 1) * BATCH_SIZE]), 'LABELS NOT CORRESPONDING REINIZIALIZE GENERATOR'
    i += 1
    if i * BATCH_SIZE >= sample_count:
      break
  
  return features


In [ ]:
MODEL_PATH = "/content/gdrive/Shareddrives/COMPUTER_VISION/models"

In [ ]:
# BEST MODEL -> inception_finetuning_classification_3_last_one_more_train_all_parameters_more_train.h5
loaded = models.load_model(os.path.join(MODEL_PATH, 'inception_finetuning_classification_3_last_one_more_train_all_parameters_more_train.h5'))
#loaded.summary()

finetuned_extactor = Model(loaded.input, loaded.layers[-2].output)
finetuned_extactor.summary()

In [ ]:
sketches_features_finetuned = extract_features(finetuned_extactor, sketches_generator, 20000)
mirflickr_features_finetuned = extract_features(finetuned_extactor, mirflickr_generator, 25000)

In [ ]:
def average_precision(requested_label, result_labels, n_ground_truth = 80):
  """
  label ricercata, label ottenute, il numero di oggetti che ci sono quella label
  """
  correct_array = (requested_label == result_labels).astype(int)
  precision_array = [np.mean(correct_array[:k]) for k in range(1, correct_array.shape[0] + 1)]
  # print(precision_array * correct_array) # mi rimangono solo quelli a 1
  return np.sum(precision_array * correct_array) / n_ground_truth


In [ ]:
def mAP(index, features, n_queries = 80, n_labels = 250, img_per_labels = 80, mode = 'euclidean'):
  """ 
  calcolo la mAP su 80 query per ogni class label 
  per poi calcolare mean(mAP) e std(mAP) per ogni classe 
  """
  counter = 0

  sum = 0
  classes_AP = []
  for label in range(n_labels): 
    counter += 1
    AP_values = []
    for i in range(img_per_labels):
      image_idx = label*n_queries + i 

      #print(image_idx)
      #if label*n_queries >= 80:
      #  # sketch image displaying 
      #  image1 = SKETCHES_DIR + '/png/alarm clock/' + str(image_idx+1) + ".png"
      #  display(Image.from_file(image1, width=229, height=229))

      res = index.query(features[image_idx], img_per_labels, mode = mode)
      assert res['labels'][0] == label, 'deve essere della stessa label'
      a = average_precision(label, res['labels'], img_per_labels)
    
      AP_values.append(a)
    classes_AP.append(AP_values)

    if counter == 2:
      return classes_AP

In [ ]:
no_index_finetuned = NO_INDEX()
no_index_finetuned.insert(np.vstack((sketches_features_finetuned, mirflickr_features_finetuned)), 
                np.concatenate((sketches_generator.filenames, mirflickr_generator.filenames)), 
                np.concatenate((sketches_generator.labels, np.array([250] * mirflickr_features_finetuned.shape[0]))))

In [ ]:
map_value = mAP(no_index_finetuned, sketches_features_finetuned)
print(map_value)

[[0.7242832081986927, 0.12363636800117044, 0.15170553829215488, 0.7519308131376713, 0.2213613124595284, 0.6205056427041493, 0.0125, 0.0125, 0.12777973223719008, 0.76151889044459, 0.0324818563789152, 0.7537407903524598, 0.5283259595856107, 0.759174779509735, 0.0125, 0.46183903012147576, 0.013257575757575756, 0.01854957805907173, 0.7096201298154987, 0.07749118308190446, 0.7684292846174299, 0.7425936480705341, 0.5496752994093665, 0.473227398236734, 0.7472451590637876, 0.1451355219851131, 0.4274054196733713, 0.06877572541236296, 0.41405455355846366, 0.030282444005270092, 0.4004845135294768, 0.34422268558554003, 0.013031914893617023, 0.7071871464634901, 0.2073718619568546, 0.1907019155929949, 0.6971478155213313, 0.23345146531428812, 0.7145870706818662, 0.2909983444379087, 0.6835951899964241, 0.04854853427654711, 0.4106624738663898, 0.7252705952866152, 0.4463462733855776, 0.6904040989445066, 0.03018433823121323, 0.5072991626790864, 0.7480729960319417, 0.759612800085049, 0.27638604736175887, 

In [ ]:
for item in map_value:
  print(max(item))
  max_index = [i for i, j in enumerate(a) if j == m]
  print(min(item))
  print(np.std(item))


0.7684292846174299
0.0125
0.2839384473291182
0.844064322282614
0.0125
0.29381322038902813
